# Baseten

[Baseten](https://baseten.co) 是 LangChain 生态系统中的一个[Provider](/docs/integrations/providers/baseten)，实现了 LLMs 组件。

此示例演示了如何使用 LangChain 以及托管在 Baseten 上的 LLM — Mistral 7B。

# 设置

要运行此示例，您需要：

* 一个 [Baseten 账户](https://baseten.co)
* 一个 [API 密钥](https://docs.baseten.co/observability/api-keys)

将您的 API 密钥导出为名为 `BASETEN_API_KEY` 的环境变量。

```sh
export BASETEN_API_KEY="paste_your_api_key_here"
```

# 单个模型调用

首先，您需要将模型部署到 Baseten。

您可以通过 [Baseten 模型库](https://app.baseten.co/explore/) 一键部署 Mistral 和 Llama 2 等基础模型，或者如果您有自己的模型，可以 [使用 Truss 进行部署](https://truss.baseten.co/welcome)。

在本示例中，我们将使用 Mistral 7B。请在此处 [部署 Mistral 7B](https://app.baseten.co/explore/mistral_7b_instruct)，并使用您部署的模型 ID（可在模型仪表板中找到）进行后续操作。

In [ ]:
##Installing the langchain packages needed to use the integration
%pip install -qU langchain-community

In [ ]:
from langchain_community.llms import Baseten

In [ ]:
# Load the model
mistral = Baseten(model="MODEL_ID", deployment="production")

In [ ]:
# Prompt the model
mistral("What is the Mistral wind?")

# 模型调用链

我们可以将对一个或多个模型的多次调用链接在一起，这正是 Langchain 的宗旨所在！

例如，在该终端仿真演示中，我们可以用 Mistral 替换 GPT。

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import PromptTemplate

template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)


chatgpt_chain = LLMChain(
    llm=mistral,
    llm_kwargs={"max_length": 4096},
    prompt=prompt,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2),
)

output = chatgpt_chain.predict(
    human_input="I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd."
)
print(output)

In [ ]:
output = chatgpt_chain.predict(human_input="ls ~")
print(output)

In [ ]:
output = chatgpt_chain.predict(human_input="cd ~")
print(output)

In [ ]:
output = chatgpt_chain.predict(
    human_input="""echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py"""
)
print(output)

正如我们从最终示例中看到的，它输出的数字可能正确也可能不正确，该模型只是在近似可能的终端输出，而不是实际执行提供的命令。尽管如此，该示例还是展示了 Mistral 充裕的上下文窗口、代码生成能力，以及在对话序列中保持主题的能力。